# 財報選股

參考資料：

* [Python 財報爬蟲](https://goo.gl/rVZhZa)
* [請問有高手會匯入這個網頁的資料到excel嗎? ](https://www.mobile01.com/topicdetail.php?f=291&t=3360713)

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def financial_statement(year, season, exchange='sii', type='綜合損益表'):
    if year > 1911:
        year -= 1911
        
    if type == '綜合損益表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析查詢彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')

    # 一些參數：
    # TYPEK => 市場別
    # sii>上市
    # otc>上櫃
    # rotc>興櫃
    # pub>公開發行

    # year => 年度
    # season => 季別
    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK': exchange,
        'year': year,
        'season': season,
    })
    
    r.encoding = 'utf8'
    
    dfs = pd.read_html(r.text)
    
    
    for i, df in enumerate(dfs):
        df.columns = df.iloc[0]
        dfs[i] = df.iloc[1:]
        
    df = pd.concat(dfs).applymap(lambda x: x if x != '--' else np.nan)
    df = df[df['公司代號'] != '公司代號']
    df = df[~df['公司代號'].isnull()]
    if any(df.columns.str.contains("合計：")):
        df = df.loc[:, ~df.columns.str.contains("合計：")]
    return df

In [3]:
df = financial_statement(2017, 2, type='營益分析查詢彙總表')

c:\users\victor\anaconda3\envs\pystock\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [4]:
df[df['公司名稱']=='台泥']

,公司代號,公司名稱,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),營業收入(百萬元),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入)
1,1101,台泥,18.01,13.08,45893.85,12.54,9.23


In [5]:
cond1 = df['毛利率(%)(營業毛利)/(營業收入)'].astype(float) > 20
cond2 = df['營業利益率(%)(營業利益)/(營業收入)'].astype(float) > 5

In [6]:
df[cond1 & cond2]

,公司代號,公司名稱,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),營業收入(百萬元),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入)
9,1203,味王公司,31.09,15.37,3076.03,15.38,11.03
13,1216,統一,33.30,6.08,196218.29,8.66,7.10
20,1227,佳格,27.73,9.41,11996.01,8.85,6.76
21,1229,聯華實業,22.24,10.61,2356.89,47.00,45.06
27,1256,鮮活果汁-KY,30.13,15.41,1422.17,15.56,9.79
28,1262,綠悅-KY,29.53,23.65,3751.43,24.02,17.86
34,1307,三芳化工,32.56,16.02,5057.42,14.70,11.59
43,1319,東陽,25.22,11.21,11962.03,12.35,9.51
45,1323,永裕,23.30,12.10,1517.63,11.23,8.55
51,1338,廣華-KY,27.04,12.24,3668.62,12.94,9.54
